## Import Initial Data

In [1]:
import pandas as pd

In [2]:
school_load = "Resources/schools_complete.csv"
student_load = "Resources/students_complete.csv"

In [3]:
school_data = pd.read_csv(school_load)
student_data = pd.read_csv(student_load)

In [4]:
data = pd.merge(student_data, school_data, how = "left", on = ["school_name", "school_name"])
data

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


## District Summary

In [5]:
schools_total = data["school_name"].nunique()

In [6]:
students_total = data["Student ID"].count()

In [7]:
budget_list = data["budget"].unique()
budget_total = (budget_list).sum()

In [8]:
math_avg = data["math_score"].mean()

In [9]:
read_avg = data["reading_score"].mean()

In [10]:
math_pass_raw = data.loc[data["math_score"] >= 70, ["math_score"]].count()
math_pass_mean = math_pass_raw.mean()
math_pass = math_pass_mean / students_total * 100

In [11]:
read_pass_raw = data.loc[data["reading_score"] >= 70, ["reading_score"]].count()
read_pass_mean = read_pass_raw.mean()
read_pass = read_pass_mean / students_total * 100

In [12]:
ovr_pass_raw = data.loc[(data["reading_score"] >= 70) & (data["math_score"] >= 70), ["reading_score"]].count()
ovr_pass_mean = ovr_pass_raw.mean()
ovr_pass = ovr_pass_mean / students_total * 100

In [13]:
district_df = pd.DataFrame([[schools_total, students_total, budget_total, math_avg, read_avg, math_pass, read_pass, ovr_pass]], 
                           columns=["Total Schools", "Total Students", "Total Budget", "Average Math Score", 
                                    "Average Reading Score", "% Passing Math", "% Passing Reading", "% Passing Overall"])
district_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [37]:
school_type = data.iloc[:, [4,8]]
df_1 = school_type.drop_duplicates()

In [38]:
df_1["Total Students"] = data.groupby(["school_name"])["student_name"].transform(len)

C:\Users\zachr\AppData\Local\Temp\ipykernel_28460\1324822495.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1["Total Students"] = data.groupby(["school_name"])["student_name"].transform(len)


In [39]:
school_budget = data.iloc[:, [4, 10]]
df_2 = school_budget.drop_duplicates

In [40]:
df_3 = pd.merge(df_1, df_2(), how = "inner", on = "school_name")

In [41]:
df_3["Total School Budget"] = (df_3["budget"]) / (df_3["Total Students"])

In [42]:
math_avg_school = data.groupby(["school_name"])["math_score"].mean

In [43]:
df_4 = pd.merge(df_3, math_avg_school(), how = "inner", on = "school_name")

In [44]:
read_avg_school = data.groupby(["school_name"])["reading_score"].mean

In [45]:
df_5 = pd.merge(df_4, read_avg_school(), how = "inner", on = "school_name")

In [46]:
math_pass_school = data.iloc[:, [4, 6]]
math_pass_school

math_pass_school_percent = math_pass_school[math_pass_school.math_score >= 70].groupby("school_name").count

In [47]:
df_6 = pd.merge(df_5, math_pass_school_percent(), how = "inner", on = "school_name")
df_6["% Passing Math"] = (df_6["math_score_y"]) / (df_6["Total Students"]) * 100


In [48]:
read_pass_school = data.iloc[:, [4, 5]]
read_pass_school

read_pass_school_percent = read_pass_school[read_pass_school.reading_score >= 70].groupby("school_name").count

In [50]:
df_7 = pd.merge(df_6, read_pass_school_percent(), how = "inner", on = "school_name")
df_7["% Passing Reading"] = (df_7["reading_score_y"]) / (df_7["Total Students"]) * 100

In [94]:
ovr_pass_school = data.iloc[:, [4, 5, 6]]

ovr_pass_school_percent = ovr_pass_school[ovr_pass_school.reading_score & ovr_pass_school.math_score >= 70].groupby("school_name").count

ovr_pass_school_percent()

,reading_score,math_score
school_name,,
Bailey High School,1510,1510
Cabrera High School,945,945
Figueroa High School,888,888
Ford High School,835,835
Griffin High School,736,736
Hernandez High School,1438,1438
Holden High School,212,212
Huang High School,861,861
Johnson High School,1437,1437


In [87]:
df_8 = pd.merge(df_7, ovr_pass_school_percent(), how = "inner", on = "school_name")
df_8["% Overall Passing"] = (df_8["reading_score"]) / (df_8["Total Students"]) * 100
df_8

,school_name,type,Total Students,budget,Total School Budget,math_score_x,reading_score_x,math_score_y,% Passing Math,reading_score_y,% Passing Reading,reading_score,math_score,% Overall Passing
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,1916,65.683922,2372,81.316421,861,861,29.516627
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,1946,65.988471,2381,80.739234,888,888,30.111902
2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,1653,93.867121,1688,95.854628,876,876,49.744463
3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,3094,66.752967,3748,80.862999,1438,1438,31.024811
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,1371,93.392371,1426,97.138965,736,736,50.136240
5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,2143,93.867718,2204,96.539641,1139,1139,49.890495
6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,1749,94.133477,1803,97.039828,945,945,50.861141
7,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,3318,66.680064,4077,81.933280,1510,1510,30.345659
8,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,395,92.505855,411,96.252927,212,212,49.648712
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,910,94.594595,923,95.945946,480,480,49.896050
